In [1]:
import numpy as np
import pandas as pd
import bs4
import requests

In [2]:
req=requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695")#getting the url

In [3]:
type(req)

requests.models.Response

In [4]:
#importing BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(req.text, 'lxml')

In [5]:
#Creating Columns for table
col= ['Postalcode','Borough','Neighborhood']
datatable=pd.DataFrame(columns=col)

In [6]:
table=soup.table.tbody #getting the tbody in html

In [7]:
#gettig all the required data
postcode = 0
borough = 0
neighborhood = 0
for tr in table.find_all('tr'):
    i=0
    for td in tr.find_all('td'):
        if i==0:
            postcode=td.text
            i=i+1
        elif i==1:
            borough=td.text
            i=i+1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    datatable = datatable.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [8]:
#cleaning data
datatable = datatable[datatable['Borough']!='Not assigned']
datatable = datatable[datatable['Borough']!=0]
datatable = datatable[datatable['Neighborhood']!='Not assigned']
datatable.reset_index(drop=True, inplace=True)

In [9]:
df = datatable.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index() 

In [10]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df.shape

(102, 3)

In [12]:
geocor=pd.read_csv(r"E:\Machine Learning\Coursera\Geospatial_Coordinates.csv")
geocor.rename(columns={"Postal Code":"Postalcode"}, inplace=True)
geocor.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
data=pd.merge(df,geocor, on='Postalcode')
data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
data.shape

(102, 5)